# Project: Dynamics of Bargaining Power

## Standard Dynamic Model 

#### Genaro, Itza, & Sonia
#### May 2021 

This code was made specifically to solve the dynamyc single objective model of principal-agent proposed in Wang. 

This aplication uses the JuMP library and the Ipopt solver to solve the optimization problrem given a value for the bargaining power.

The use uf this library allows us to easily change the parametrization of the problem and rely on a robust solver to find solutions in multi-criteria non-linear restricted optimization problems. 

### Libraries

In [1]:
using JuMP, Ipopt # Loading optimization libraries.
                  # Ipopt solve a nonlinear optimization problem.
using DataFrames  # Loading frame libraries.
using ExcelFiles, XLSX  # Loading excel files libraries.

### Utility Functions

In [2]:
#v(c,a,g=1,alpha=1)=-exp(g*(a-alpha*c))  # Declaration of v (agent utility function); 
                                        # g is the coefficient of risk aversion
                                        # alpha is a cost coefficientv

v(c, a, h=0.75) =  c^(1-h)/(1-h)- a^2  #CRA Function 
                                    #h is a oefficient of risk aversion
                                    # a is the agent effort

u(y,w)=y-w  #Declaration of the principal utility function 
            #y is the current output
            #w is the salary paid to the agent

u (generic function with 1 method)

### Variables and Lists

In [3]:
A = [0.1 0.2]    # Actions set [al ah]
Y = [0.4 0.8]    # Outcomes set [yl yh]
f = [2/3 1/3; 1/3 2/3]    # Probability matrix  [yh|ah yl|ah; yh|al yl|al]
beta = 0.96    # Future discount factor
N = 100    # Number of intervals for the state variable
Max_iter = 250 #Max number of iterations 
digits_tol = 3 #Digits of precision of the solution  
VFH=zeros(100)
VFL=zeros(100)
for k in K
    VFH[k]=90
    VFL[k]=19
end
iter=0

3

### High Effort Model Specification

In [ ]:
function Model_High_Effort(k, P, Q, h, V, U0)
    """
    This functions declares and solves the High effort model for the principal utility
    Returns optimal values if found 
    """
    model_high_effort = Model(with_optimizer(Ipopt.Optimizer, tol = 1e-7, max_iter = 1000, print_level=1)) 
    # A model to incentivize high effort is created specifying the optimization method

    #variables del modelo xl=wl, xh=wh
    @variable(model_high_effort, xl>=0.00001) 
    @variable(model_high_effort, xh>=0.00001)

    #Declare the agent's utility function within the model.

    register(model_high_effort, :v, 3, v, autodiff=true) # model_high_effort, the model name
                                            # :v,  **********                                          
                                            # 2, number of variables
                                            # v, declare the agent's utility function 
                                            # autodiff,    **********
    
    register(model_high_effort, :u, 2, u, autodiff=true) # model_high_effort, the model name
                                            # :u,  **********                                          
                                            # 2, number of variables
                                            # u, declare the principal's utility function 
                                            # autodiff,    **********

    #Non-linear type expressions are declared, the expected utility of the agent, within the model.
    EV_H = @NLexpression(model_high_effort, f[1,1]*(v(xh,A[2],h)+beta*V[P])+f[1,2]*(v(xl,A[2],h)+beta*V[Q])) #E(V|ah)
    EV_L = @NLexpression(model_high_effort, f[2,1]*(v(xh,A[1],h)+beta*V[P])+f[2,2]*(v(xl,A[1],h)+beta*V[Q])) #E(V|al)
    EU_H = @NLexpression(model_high_effort,f[1,1]*(u(Y[2],xh)+beta*U0[P])+f[1,2]*(u(Y[1],xl)+beta*U0[Q])) #E(U|ah)
    
    #Objective function; expected utility of the principal given ah.
    @NLobjective(model_high_effort, Max, EU_H) 

    @constraint(model_high_effort, xh<=Y[2]) #Financial capacity restriction for high compensation; wh<=yh.
    @constraint(model_high_effort, xl<=Y[1])  #Financial capacity restriction for low compensation; wl<=yl.
    @NLconstraint(model_high_effort, EV_H>=EV_L) #Incentive constraint to incentivize ah.
    @NLconstraint(model_high_effort, EV_H == V[k]) #Participation constraint to incentivize ah. 

    #The problem posed for incentive ah is solved.
    JuMP.optimize!(model_high_effort) 
    
    # The optimal value of the function is saved given ah.
    uh = getobjectivevalue(model_high_effort) 
    stat = termination_status(model_high_effort)
    wl = value(xl)
    wh = value(xh)
    
    return uh, wl, wh, stat
end

### Low Effort Model Specification 

In [ ]:
function Model_Low_Effort(k,P,Q,h, V, U0)
    """
    This functions declares and solves the Low effort model for the principal utility
    Returns optimal values if found 
    """
    
    # A optimization model is created to incentive al.
    model_low_effort = Model(with_optimizer(Ipopt.Optimizer, tol = 1e-7, max_iter = 1000, print_level=1)) 

    #Declare the decision variables for the model: xl = wl, xh = wh.
    @variable(model_low_effort, xl>=0.00001) 
    @variable(model_low_effort, xh>=0.00001)

    #Declare the agent's utility function within the model.
    register(model_low_effort, :v, 3, v, autodiff=true) # model_low_effort, the model name
                                            # :v,  **********                                          
                                            # 3, number of variables
                                            # v, declare the agent's utility function 
                                            # autodiff,    **********
    register(model_low_effort, :u, 2, u, autodiff=true)# model_low_effort, the model name
                                            # :u,  **********                                          
                                            # 2, number of variables
                                            # u, declare the principal's utility function 
                                            # autodiff,    **********
    
    # Non-linear type expressions are declared, the expected utility of the agent, within the model.
    
    EV_H = @NLexpression(model_low_effort, f[1,1]*(v(xh,A[2],h)+beta*V[P])+f[1,2]*(v(xl,A[2],h)+beta*V[Q])) #E(V|ah)
    EV_L = @NLexpression(model_low_effort, f[2,1]*(v(xh,A[1],h)+beta*V[P])+f[2,2]*(v(xl,A[1],h)+beta*V[Q])) #E(V|al)
    EU_L = @NLexpression(model_low_effort,f[2,1]*(u(Y[2],xh)+beta*U0[P])+f[2,2]*(u(Y[1],xl)+beta*U0[Q])) # E(u|al))
    
    # Objective function; expected utility of the principal given al.l
    @NLobjective(model_low_effort, Max, EU_L) 

    @constraint(model_low_effort, xh <= Y[2]) # Financial capacity restriction for high compensation; wh<=yh.
    @constraint(model_low_effort, xl <= Y[1]) # Financial capacity restriction for low compensation; wl<=yl.
    
    @NLconstraint(model_low_effort, EV_H <= EV_L)  # Incentive constraint to incentivize al.
    @NLconstraint(model_low_effort, EV_L == V[k])  # Participation constraint to incentivize al.

    # The problem posed for incentive al is solved.
    JuMP.optimize!(model_low_effort)
    
    # The optimal value of the function is saved given al.
    ul=getobjectivevalue(model_low_effort) #Se guarda el valor optimo de la funcion para al
    stat = termination_status(model_low_effort)
    wl = value(xl)
    wh = value(xh)
    
    return ul, wl, wh, stat
end

### Feasible Indexes 

In [ ]:
function feasible_Indexes(h, V, U0)
    """
    This functions finds the indexes where the principal optimization problem has a feasible solution.
    Returns array of feasible indexes. 
    """
    K0 = 1:10 #Initial guess of possible feasible indexes
    KN = 1:100 #Final set of possible feasible indexes
    
    #loop while the initial set and the final set of indexes are different 
    while K0 != KN 
    
        K0 = KN 
        KN = []

        for k in K0 #loop for each index in the set

            fact = 0 #factibility indicator 

            for P in k:min(K0[end], Int(VFH[k])+3) # Loop for comparison value 1

                for Q in max(K0[1],Int(VFL[k]-3)):k # Loop for comparison value 2

                    stat_high = Model_High_Effort(k, P, Q, h, V, U0)[4] #Solve the model for high effort, return termination status
        
                    if (stat_high == MOI.LOCALLY_SOLVED) #check if the model for high effort on index k if feasible, if so exit the loopfor comprasion value 2 
                        fact=1 
                        break    
                    end

                    stat_low = Model_Low_Effort(k,P,Q, h, V, U0)[4] #Solve the model for low effort, return termination status

                    if (stat_low == MOI.LOCALLY_SOLVED) #check if the model for low effort on index k if feasible, if so exit the loopfor comprasion value 2 
                        fact = 1
                        break
                    end

                end
                
                #if the index is feasible, break the loop for comprasion value 1 
                if fact == 1 
                    break
                end

            end
            
            #if the index is feasible, append it on the final set
            if fact == 1
                append!(KN, k)
            end

        end

        print(KN, "\n")

    end
    
    #Return the final set of feasible indexes
    return KN 
    
end

### Routine 

In [ ]:
function Dynamic_Pareto_Frontier(K, V, U0, UF, digits_tol, Max_iter, h)
    """
    This functions finds the stationary point of the Bellman Equation for
    the standard dynamic model.
    Returns optimal values if found 
    """
    
    iter=0 #Current number of method iteration 
    size_iter = length(K) #Size of the iteration 
    AcOpt = [] # List of optimal actions for K
    WHOpt = [] # List of hight optimal compensation for K
    WLOpt = [] # List of low optimal compensation for K
    VFH=zeros(100) #List of promised discounted utility for the principal if yh occurs.
    VFL=zeros(100)# List of promised discounted utility for the principal if yl occurs.
    for k in K
        VFH[k]=K[end]
        VFL[k]=K[1]
    end
    #loop while the initial value for the state variable is different from it's final value. 
    while round.(U0[K],digits = digits_tol)!=round.(UF[K],digits = digits_tol)
    
        AcOpt=[] #List of recomended actions 
        WHOpt=[] #List of compensations if yh occurs
        WLOpt=[] #List of compensations if yl occurs 
        U0[K]=UF[K] #Update state variable
        UF=zeros(100) #Final value of the state variable 

        for k in K # For each k in the feasible index set

            uval=0 #Value of the principal's current discounted utility 
            wl=0 #Low compensation 
            wh=0 #High compendation 
            accopt=0 #Recomended action 
            p=0 #Optimal comprasion value 1 
            q=0 #Optimal comprasion value 2 
 
            for P in k:min(K[end], Int(VFH[k])+3) # Loop for comparison value 1

                for Q in max(K[1],Int(VFL[k]-3)):k # Loop for comparison value 2

                    uh, xl, xh, stat_high = Model_High_Effort(k, P, Q, h, V, U0) #High Effort Model is Solved
                    
                    # If the principal's utility with the high effort action is higher than 0 and the model is feasible update the state variables
                    if (uh>uval) & (stat_high == MOI.LOCALLY_SOLVED)
                        uval=uh
                        wh=xh
                        wl=xl
                        p=P
                        q=Q
                        accopt=A[2]
                       
                    end

                    ul, xl, xh, stat_low = Model_Low_Effort(k, P, Q, h, V, U0)
                    
                    # If the principal's utility with the low effort action is higher than with the low effort and the model is feasible update the state variables
                    if (ul>uval) & (stat_low == MOI.LOCALLY_SOLVED)
                        uval=ul
                        wh=xh
                        wl=xl
                        p=P
                        q=Q
                        accopt=A[1]
                
                    end

                end
            end
            
            #The new values for the state variables are saved
            UF[k]=uval 
            append!(WHOpt, wh) 
            append!(WLOpt,wl)
            append!(AcOpt, accopt)
            VFH[k]=p
            VFL[k]=q
            
            if iter<=1 
                print("K= ", k, "  ul=", uval,"   wh=", wh, "   wl=", wl, "  P=",p,"  Q=",q,"  ", accopt, "\n")
            end
        end
        
        iter+=1 #Update number of iterations 
        println(iter)
        
        if iter == Max_iter #if the number of iterations is higher than the Max-iter, stationary point could not be found 
            break
        end

    end
    
    #Creation of the DataFrame
    DataEstPoint = DataFrame(
    K=K,
    Utilidad_Agente=V[K],
    Utilidad_Principal=U0[K],
    Compensacion_YH=WHOpt,
    Compensacion_YL=WLOpt,
    Accion_Recomendada=AcOpt, 
    VFH=VFH[K], 
    VFL=VFL[K])
    
    return DataEstPoint
end

### Loop for diferent values of h.

In [ ]:
H = [0.44,0.45,0.46,0.47,0.48,0.49]
for h in H  
    vl = v(0,A[2], h)/(1-beta) # State variable`s lower limit 
    vh = ((1/3)*v(Y[1],A[1],h)+(2/3)*v(Y[2],A[1],h))/(1-beta)  # State variable's upper limit
    V = LinRange(vl, vh, N) # State variable's vector 
    UF = zeros(100) # List of final values of the principal utility
    Din = DataFrame(XLSX.readtable(string("DataEstPointh",repr(round.(h-0.01,digits = 2)),".xlsx"), "Sheet1")...)
    K_last = Din[:,"K"]
    UF[K_last] = Din[:,"Utilidad_Principal"]
    U0 = ones(100) # State variable's initial value
    K =  feasible_Indexes(h, V, U0) #Find feasible indexes for a given value of h 
    DataEstPoint = Dynamic_Pareto_Frontier(K, V, U0, UF, digits_tol, Max_iter, h) #Run the routine to find the stationary point
    save(string("DataEstPointh",repr(h),".xlsx"), DataEstPoint) #Save the dataframe to an excel file. 
end

### Animation Plot 

#### Libraries 

In [ ]:
using Plots, Plots.PlotMeasures    # Plots packages load
using DataFrames
using ExcelFiles
using XLSX

#### Plotting

#### Pareto Frontier Animation 

In [ ]:
theme(:ggplot2)     # Graphics theme

anim = @animate for h_now = 10:99
    Data_h = DataFrame(XLSX.readtable(string("DataEstPointh",repr.(round(h_now/100,digits = 2)),".xlsx"), "Sheet1")...)
    Principal_Utilities = Data_h[4:end,"Utilidad_Principal"]
    K = Data_h[4:end,"K"]
    plot(K, Principal_Utilities, 
    xlabel = "Agent reservation utility, level k",
    ylabel =  "Principal's optimal expected utility",
    title="Pareto Frontier for different levels of h.",
    labels = string("h=",repr(h_now/100)),
    legend = :topright,
    ylims=(0,20), 
    xlims=(0,100))
    
end
 
gif(anim, "Principal_Utilities_varying_h_Dynamic.gif", fps = 5)

#### Compensation Scheme Animation 

In [ ]:
theme(:ggplot2)     # Graphics theme

anim = @animate for h_now = 10:99
    Data_h = DataFrame(XLSX.readtable(string("DataEstPointh",repr.(round(h_now/100,digits = 2)),".xlsx"), "Sheet1")...)
    High_Compensations = Data_h[4:end,"Compensacion_YH"]
    Low_Compensations = Data_h[4:end,"Compensacion_YL"]
    K = Data_h[4:end,"K"]
    scatter(K, [Low_Compensations, High_Compensations], 
    shape=[:hline :+ :utri],
    markersize=8,
    title="Compensation for different levels of h.", 
    legend=:topleft,
    label=[string("WL, h=",repr(h_now/100)) string("Wh, h=",repr(h_now/100)) ],
    xlabel = "Agent reservation utility, level k",
    ylabel="Compensation",
    ylims=(0,0.8), 
    xlims=(10,100))
    
end
 
gif(anim, "Dynamic_Compensation_varying_h.gif", fps = 5)

In [ ]:
theme(:ggplot2)     # Graphics theme

anim = @animate for h_now = 10:99
    Data_h = DataFrame(XLSX.readtable(string("DataEstPointh",repr.(round(h_now/100,digits = 2)),".xlsx"), "Sheet1")...)
    VFH = Data_h[4:end,"VFH"]
    VFL = Data_h[4:end,"VFL"]
    K = Data_h[4:end,"K"]
    scatter(K, [VFL, VFH], 
    shape=[:hline :+ :utri],
    markersize=8,
    title="Promised Discounted Utility for different levels of h.", 
    legend=:topleft,
    label=[string("VFL, h=",repr(h_now/100)) string("VFH, h=",repr(h_now/100)) ],
    xlabel = "Agent reservation utility, level k",
    ylabel="Promised Discounted Utility",
    ylims=(0,100), 
    xlims=(0,100))
    
end
 
gif(anim, "Promised_Utility_varying_h.gif", fps = 5)